In [93]:
import os
import json
import nest_asyncio
from llama_index.core import VectorStoreIndex
from llama_index.core.schema import TextNode

# Apply nest_asyncio
nest_asyncio.apply()

# Path to the JSON file
questions_json_path = r"C:\Users\localadmin\Desktop\new desktop\linkedinsucks\linkedin_easyapply_bot\all_data\docs\questions.json"

# Load JSON data and create TextNode objects
few_shot_nodes = []
for line in open(questions_json_path, "r"):
    few_shot_nodes.append(TextNode(text=line))

# Create a VectorStoreIndex and retriever
few_shot_index = VectorStoreIndex(few_shot_nodes)
few_shot_retriever = few_shot_index.as_retriever(similarity_top_k=2)

# Function to extract key-value pairs from a string
def extract_key_value_from_string(input_string):
    # Remove any trailing commas and newline characters
    cleaned_string = input_string.strip().rstrip(',\n}{')
    if ':' not in cleaned_string:
        return "", ""
    # Split the string into key and value
    key, value = cleaned_string.split('": ')
    
    # Remove any surrounding quotes from key and value
    key = key.strip('"')
    value = value.strip('"')
    
    return key, value

# Function to generate few-shot examples
def few_shot_examples_fn(query_str):
    retrieved_nodes = few_shot_retriever.retrieve(query_str)
    result_strs = []
    for n in retrieved_nodes:
        query, response = extract_key_value_from_string(n.get_content())
        result_str = f"Query: {query}\nResponse: {response}"
        result_strs.append(result_str)
    return "\n\n".join(result_strs)
